In [1]:
import sys
sys.path.append('..')

In [2]:
FRMA_PLATFORMS = ['GPL' + str(x) for x in [
    570, 571, 96,
    1261,  # organism Mus musculus
    6244,  # getting strange errors. data in matrix
    # 1355,  # NO FRMA, Rattus norvegicus,
    6246,  # no frma package (?). read.affybatch: The affy package can process data from the Gene ST 1.x series of arrays,
    # but you should consider using either the oligo or xps packages, which are specifically
    # designed for these arrays.
    # there is no package called ‘mogene10stv1frmavecs’

    90, # organism Saccharomyces cerevisiae
    3921,
    # 5175, # no frma package (?). read.affybatch: The affy package is not designed for this array type. 
    # Please use either the oligo or xps package.
    # 5188, # no cdf frma packages
    8321,  # organism Mus musculus
]]

AGLIENT_2CH_PLATFORMS = ['GPL4133', 'GPL6480', 'GPL15849', 'GPL1708', 'GPL887', 'GPL4091', 'GPL9128', 'GPL7264', 'GPL11387',
             'GPL8687', 'GPL6848', 'GPL2879', 'GPL5477', 'GPL8841', 'GPL10123', 'GPL4093', 'GPL11386', 'GPL10806',
             'GPL8269', 'GPL10150', 'GPL8583', 'GPL15931', 'GPL4126', 'GPL10152', 'GPL16050', 'GPL2567', 'GPL9053',
             'GPL14550', 'GPL5325', 'GPL10808', 'GPL13691', 'GPL9075', 'GPL8736', 'GPL885', 'GPL9777', 'GPL7504',
             'GPL8693', 'GPL2873', 'GPL17077', 'GPL10734', 'GPL13953', 'GPL13607', 'GPL13685', 'GPL7015', 'GPL15560',
             'GPL18623', 'GPL10481', 'GPL16280', 'GPL8737', 'GPL11068']

ILLUMINA_PLATFORMS = list(map('GPL{}'.format, [
   10558, 6947
]))

In [3]:
from lib.annotation_api import Api

In [4]:
api = Api('scraper_test_dev')

In [5]:
import pandas as pd

In [11]:
frma_not_processed = pd.DataFrame(list(api.search_samples(platform=FRMA_PLATFORMS, preprocessed_exists=0)))
frma_not_processed.head()

,accession,annotation,channel,characteristics,characteristics_raw,data_source,description,extracted_molecule,instrument_model,label,...,protocols,relations,sample_type,scrap_date,series,source_name,status,submission_date,supplementary_files,title
0,GSM1690912,NaN,NaN,"{'genotype': 'ighv unknown myd88 wt', 'age': '...","[tissue: Peripheral Blood, genotype: IGHV unkn...",geo,[Gene expression data from CLL patients],total RNA,NaN,Biotin,...,"[{'type': 'data_processing_protocol', 'text': ...",{},RNA,2015-09-22 18:11:15.749,[GSE69034],CLL Cells,Public on Jun 01 2015,2015-05-19,[],CLL Patient Sample 49_CW210
1,GSM1690914,NaN,NaN,"{'genotype': 'ighv unknown myd88 wt', 'age': '...","[tissue: Peripheral Blood, genotype: IGHV unkn...",geo,[Gene expression data from CLL patients],total RNA,NaN,Biotin,...,"[{'type': 'data_processing_protocol', 'text': ...",{},RNA,2015-09-22 18:11:15.773,[GSE69034],CLL Cells,Public on Jun 01 2015,2015-05-19,[],CLL Patient Sample 82_CW258
2,GSM1690909,NaN,NaN,"{'genotype': 'ighv unmutated myd88 wt', 'age':...","[tissue: Peripheral Blood, genotype: IGHV unmu...",geo,[Gene expression data from CLL patients],total RNA,NaN,Biotin,...,"[{'type': 'data_processing_protocol', 'text': ...",{},RNA,2015-09-22 18:11:15.717,[GSE69034],CLL Cells,Public on Jun 01 2015,2015-05-19,[],CLL Patient Sample JB2009091005
3,GSM1690911,NaN,NaN,"{'genotype': 'ighv unmutated myd88 wt', 'age':...","[tissue: Peripheral Blood, genotype: IGHV unmu...",geo,[Gene expression data from CLL patients],total RNA,NaN,Biotin,...,"[{'type': 'data_processing_protocol', 'text': ...",{},RNA,2015-09-22 18:11:15.739,[GSE69034],CLL Cells,Public on Jun 01 2015,2015-05-19,[],CLL Patient Sample JB2009091022
4,GSM1690809,NaN,NaN,"{'genotype': 'ighv mutated myd88 wt', 'age': '...","[tissue: Peripheral Blood, genotype: IGHV muta...",geo,[Gene expression data from CLL patients],total RNA,NaN,Biotin,...,"[{'type': 'data_processing_protocol', 'text': ...",{},RNA,2015-09-22 18:11:14.532,[GSE69034],CLL Cells,Public on Jun 01 2015,2015-05-19,[],CLL Patient Sample JB2009091021


In [17]:
frma_not_processed.accession.unique().shape

(109980,)

In [19]:
sum(map(lambda x: 1, api.search_samples(platform=AGLIENT_PLATFORMS, preprocessed=1)))

23521

In [51]:
from sh import rsync

def get_series_for_search(api, **kvargs):
    series = {
        (se, s['platform'])
        for s in api.search_samples(**kvargs)
        for se in s['series']
    }
    return series
    
def download_series(series, platform, dest):
    target = 'npryanichnikov@ui2.computing.kiae.ru:/s/ls2/groups/g0064/preproc/data/{}_{}/out/'.format(
        series, platform
    )
    series_platform = '{}_{}'.format(series, platform)
    !mkdir "$dest/$series_platform"
    !rsync -avz "$target" "$dest/$series_platform"
#     for l in rsync(target, dest, a=True, v=True, z=True, _iter=True):
#         print(l)
    

In [8]:
series_illumina = list(api.search_series(platform=ILLUMINA_PLATFORMS, preprocessed=1))

In [10]:
len(series_illumina)

716

In [11]:
with open('ill.list', 'w') as f:
    t = '\n'.join(["/s/ls2/groups/g0064/preproc/data/{}_{}".format(series['accession'], platform) 
                   for series in series_illumina
                   for platform in series['platforms']
                   if platform in ILLUMINA_PLATFORMS])
    f.write(t)
    #download_series(series, platform, '/data/rawdata/agil_datasets')

In [12]:
!scp /home/ubuntu/python-notebooks/preproc/ill.list npryanichnikov@ui2.computing.kiae.ru:. 

ill.list                                      100%   36KB  35.8KB/s   00:00    


In [26]:
agil_series = get_series_for_search(api, platform=AGLIENT_PLATFORMS, preprocessed=1)

In [27]:
len(agil_series)

967

In [28]:
next(iter(agil_series))

('GSE19324', 'GPL4133')

In [52]:
download_series('GSE19324', 'GPL4133', '/data/rawdata/')

mkdir: cannot create directory ‘/data/rawdata//GSE19324_GPL4133’: No space left on device
receiving incremental file list
rsync: mkdir "/data/rawdata//GSE19324_GPL4133" failed: No space left on device (28)
rsync error: error in file IO (code 11) at main.c(674) [Receiver=3.1.0]


In [45]:
!mkdir /data/rawdata/agil_datasets

In [111]:
pwd

'/home/ubuntu/python-notebooks/preproc'

In [105]:
with open('agil.list', 'w') as f:
    t = '\n'.join(["/s/ls2/groups/g0064/preproc/data/{}_{}".format(series, platform) for series, platform in agil_series])
    f.write(t)
    #download_series(series, platform, '/data/rawdata/agil_datasets')

In [110]:
!scp -P 15166 /home/ubuntu/python-notebooks/preproc/agil.list lpenguin@hpc.mipt.ru:. 

The authenticity of host 'hpc.mipt.ru (93.175.29.144)' can't be established.
RSA key fingerprint is 03:17:31:07:b4:13:20:0e:db:9e:dd:56:00:78:43:3a.
Are you sure you want to continue connecting (yes/no)? ^C


In [113]:
!scp /home/ubuntu/python-notebooks/preproc/agil.list npryanichnikov@ui2.computing.kiae.ru:. 

agil.list                                     100%   47KB  47.2KB/s   00:00    


In [101]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
/dev/xvda1      7.8G  6.2G  1.3G  84% /
none            4.0K     0  4.0K   0% /sys/fs/cgroup
udev            2.0G   12K  2.0G   1% /dev
tmpfs           396M  368K  395M   1% /run
none            5.0M   12K  5.0M   1% /run/lock
none            2.0G     0  2.0G   0% /run/shm
none            100M   12K  100M   1% /run/user
/dev/xvdi       197G  146G   41G  79% /data
/dev/xvdg       4.8G   10M  4.6G   1% /journal
/dev/xvdh       4.8G  136M  4.4G   3% /log


In [63]:
rm -rf /data/rawdata/SRR361238.sra

In [95]:
!ls -1d /data/rawdata/* | grep -v cache | xargs du -sh

68K	/data/rawdata/classified_datasets_by_accession.json
2.1M	/data/rawdata/classified_samples.json
1.3M	/data/rawdata/datasets.json
176K	/data/rawdata/datasets.tar.gz
328K	/data/rawdata/Disease-all.csv
104K	/data/rawdata/Disease.json
112K	/data/rawdata/Dis.ipynb
du: cannot access ‘/data/rawdata/DO’: No such file or directory
du: cannot access ‘remote.ipynb’: No such file or directory
4.0K	/data/rawdata/end.date
4.0K	/data/rawdata/features.ipynb
18G	/data/rawdata/GEO.all.platforms
15M	/data/rawdata/graph-tool-2.2.42.tar.bz2
4.0K	/data/rawdata/in
17G	/data/rawdata/metamap
8.0K	/data/rawdata/nm.ipynb
4.0K	/data/rawdata/nm.list
4.0K	/data/rawdata/nm-samples.list
4.0K	/data/rawdata/nm-sra.list
0	/data/rawdata/ok
8.6M	/data/rawdata/out
190M	/data/rawdata/p
2.3G	/data/rawdata/public_mm_linux_main_2014.tar.bz2
815M	/data/rawdata/python-notebooks
60K	/data/rawdata/samples-100.csv
21M	/data/rawdata/samples-all.csv
1.6M	/data/rawdata/samples_class.csv
146M	/data/rawdata/scraper-dump.tar
101M	/dat

In [100]:
!rm -rf  /data/rawdata/metamap/

In [56]:
!ls -lah  /data/

total 57G
drwxr-xr-x  6 mongodb       mongodb       4.0K Oct 27 14:44 .
drwxr-xr-x 26 root          root          4.0K Oct 23 18:46 ..
-rw-------  1 mongodb       mongodb        64M Jun  5 14:01 admin.0
-rw-------  1 mongodb       mongodb        16M Jun  5 14:01 admin.ns
drwxr-xr-x  3 elasticsearch elasticsearch 4.0K Jun  5 17:01 elasticsearch
drwxr-xr-x  2 mongodb       mongodb       4.0K Oct 27 14:44 journal
-rw-------  1 mongodb       mongodb        64M Oct 27 14:43 local.0
-rw-------  1 mongodb       mongodb        16M Oct 27 14:43 local.ns
-rwxr-xr-x  1 mongodb       mongodb          6 Oct 27 14:43 mongod.lock
-rw-------  1 mongodb       nogroup        64M Oct 27 19:44 preproc.0
-rw-------  1 mongodb       nogroup        16M Oct 27 19:44 preproc.ns
drwxr-xr-x 14 ubuntu        ubuntu        4.0K Oct 27 19:29 rawdata
-rw-------  1 mongodb       nogroup        64M Jun 18 15:33 scraper_class.0
-rw-------  1 mongodb       nogroup        16M Jun 18 15:33 scraper_class.ns
-rw-------  1 m

In [54]:
!du -sh /data/rawdata/

^C


In [38]:
!rm //data/rawdata/snapshots/mongo/samples.tmp.json